Author: Chris J. Dallmann \
Affiliation: University of Wuerzburg \
Last revision: 11-March-2025

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from caveclient import CAVEclient

In [2]:
def hl_nt_dict():
    
    hl_nt_dict = {
        '0A': 'GABA',
        '0A/0B': 'GABA',
        '1A': 'ACH',
        '1B': 'GABA',
        '2A': 'GLUT',
        '3A': 'ACH',
        '3B': 'GABA',
        '4B': 'ACH',
        '5B': 'GABA',
        '6A': 'GABA',
        '6B': 'GABA',
        '7B': 'ACH',
        '8A': 'GLUT',
        '8B': 'ACH',
        '9A': 'GABA',
        '9B': 'GLUT',
        '10B': 'ACH',
        '11A': 'ACH',
        '11B': 'GABA',
        '12A': 'ACH',
        '12B': 'GABA',
        '13A': 'GABA',
        '13B': 'GABA',
        '14A': 'GLUT',
        '15B': 'GLUT',
        '16B': 'GLUT',
        '17A': 'ACH',
        '18B': 'ACH',
        '19A': 'GABA',
        '19B': 'ACH',
        '20A': 'ACH',
        '21A': 'GLUT', 
        '22A': 'ACH',
        '23B': 'ACH',
        '24B': 'GLUT'}
    
    return hl_nt_dict

In [3]:
# Select cell type
#cell_type = ['claw_flx','claw_ext']
cell_type = ['hook_flx','hook_ext']

In [4]:
# Initialize CAVEclient
# This step requires an authentication token, see https://caveclient.readthedocs.io/en/latest/guide/authentication.html
client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.materialize.version = 840

In [5]:
# Get IDs of FeCO neurons from annotation table  
df_feco = client.materialize.query_table('feco_axons_v0', filter_in_dict = {'cell_type': cell_type})

# Exclude partially reconstructed claw axons
pt_position_to_exclude = np.array([[9093, 101264, 3740], [9401, 100947, 3740]])
index_to_exclude = []
for index, _ in df_feco.iterrows():
    if (df_feco.pt_position[index]==pt_position_to_exclude).all(axis=1).any():
        index_to_exclude.append(index)
df_feco = df_feco.drop(index_to_exclude)

# Get IDs 
feco_ids = df_feco.pt_root_id.tolist()

In [6]:
# Query input synapses 
df_syn = client.materialize.synapse_query(post_ids = feco_ids)

In [7]:
# Remove connections less than syn_thresh  
syn_thresh = 3;
df_syn['syn_count'] = 1
df_syn = df_syn.groupby(['pre_pt_root_id','post_pt_root_id'])['syn_count'].sum().reset_index().sort_values('syn_count', ascending=False)
df_syn = df_syn[df_syn.syn_count>=syn_thresh]

In [8]:
# Get IDs of partner neurons
partner_ids = df_syn.pre_pt_root_id.unique().tolist()

In [9]:
# Load annotation table with hemilineages of partner neurons 
df_annotation = client.materialize.query_table('upstream_feco_axons')
annotation_ids = df_annotation.pt_root_id.tolist()

In [10]:
# Determine number of cholinergic (ACH), GABAergic (GABA), and glutamatergic (GLUT) synapses 
n_syn_nt = {}
n_syn_nt['ACH'] = 0
n_syn_nt['GABA'] = 0
n_syn_nt['GLUT'] = 0
n_syn_nt['Unknown'] = 0

nt_dict = hl_nt_dict()
nt_dict['hook'] = 'ACH'
nt_dict['club'] = 'ACH'
nt_dict['hair_plate'] = 'ACH'
nt_dict['descending'] = 'Unknown'

for partner_id in partner_ids:
    # Get number of synapses
    n_syn = df_syn.syn_count[df_syn.pre_pt_root_id == partner_id].sum().item()
    # Get hemilineage of partner neuron
    if partner_id in annotation_ids:
        cell_type = df_annotation.cell_type[df_annotation.pt_root_id == partner_id].item()
        # Look up primary neurotransmitter of hemilineage 
        nt = nt_dict[cell_type]
    else: 
        nt = 'Unknown'

    if nt == 'Unknown':
        n_syn_nt['Unknown'] += n_syn
    else:
        n_syn_nt[nt] += n_syn    

In [11]:
n_syn_nt

{'ACH': 142, 'GABA': 911, 'GLUT': 13, 'Unknown': 6}

In [16]:
n_syn_nt['GABA']/df_syn.syn_count.sum().item()

0.8498134328358209